# Road Test Batch Sequential Design

In [1]:
import ema_workbench
ema_workbench.__version__

/Users/jpn/anaconda/envs/EMAT/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


'2.1.CS1'

In [2]:
import emat
emat.__version__

'0.1.3'

In [3]:
import plotly.express as px

In [4]:
import os, numpy, pandas, functools
from matplotlib import pyplot as plt
from emat.util.xmle import Show

In [5]:
logger = emat.util.loggers.log_to_stderr(20, True)

In [6]:
road_test_scope_file = emat.package_file('model','tests','road_test_2d.yaml')

In [7]:
s = emat.Scope(road_test_scope_file)
s

<emat.Scope with 11 constants, 1 uncertainties, 1 levers, 7 measures>

In [8]:
s.info()

name: Road Test 2d
desc: EMAT Road Test Example using only 2 dimensions
constants:
  free_flow_time = 60
  initial_capacity = 100
  alpha = 0.15
  beta = 4.0
  value_of_time = 0.01
  unit_cost_expansion = 100
  amortization_period = 30
  debt_type = GO Bond
  interest_rate_lock = False
  interest_rate = 0.03
  yield_curve = 0.01
uncertainties:
  input_flow = 80 to 250
levers:
  expand_capacity = 0.0 to 100.0
measures:
  no_build_travel_time
  build_travel_time
  time_savings
  value_of_time_savings
  net_benefits
  cost_of_capacity_expansion
  present_cost_expansion


In [9]:
db = emat.SQLiteDB()

[00:22.30] MainProcess/INFO: running script scope.sql
[00:22.31] MainProcess/INFO: running script exp_design.sql
[00:22.33] MainProcess/INFO: running script meta_model.sql


In [10]:
s.store_scope(db)

In [11]:
from emat.experiment.experimental_design import design_experiments

In [12]:
design = design_experiments(
    s, 
    db=db, 
    n_samples_per_factor=10,     
    #n_samples=100, 
    sampler='lhs',
)
design.head()

,expand_capacity,input_flow,free_flow_time,initial_capacity,alpha,beta,value_of_time,unit_cost_expansion,amortization_period,debt_type,interest_rate_lock,interest_rate,yield_curve
experiment,,,,,,,,,,,,,
1,17.805981,114,60,100,0.15,4.0,0.01,100,30,GO Bond,False,0.03,0.01
2,67.515416,214,60,100,0.15,4.0,0.01,100,30,GO Bond,False,0.03,0.01
3,10.068842,166,60,100,0.15,4.0,0.01,100,30,GO Bond,False,0.03,0.01
4,83.864133,199,60,100,0.15,4.0,0.01,100,30,GO Bond,False,0.03,0.01
5,74.413206,236,60,100,0.15,4.0,0.01,100,30,GO Bond,False,0.03,0.01


## Core Model

In [13]:
from emat.model.core_python import PythonCoreModel
from emat.model.core_python import Road_Capacity_Investment

In [14]:
m = PythonCoreModel(Road_Capacity_Investment, scope=s, db=db)

In [15]:
lhs_results = m.run_experiments(design_name='lhs')
lhs_results.head()

[00:29.08] MainProcess/INFO: performing 20 scenarios/policies * 1 model(s) = 20 experiments
[00:29.10] MainProcess/INFO: performing experiments sequentially
[00:29.11] MainProcess/INFO: 2 cases completed
[00:29.11] MainProcess/INFO: 4 cases completed
[00:29.11] MainProcess/INFO: 6 cases completed
[00:29.11] MainProcess/INFO: 8 cases completed
[00:29.12] MainProcess/INFO: 10 cases completed
[00:29.12] MainProcess/INFO: 12 cases completed
[00:29.13] MainProcess/INFO: 14 cases completed
[00:29.13] MainProcess/INFO: 16 cases completed
[00:29.14] MainProcess/INFO: 18 cases completed
[00:29.14] MainProcess/INFO: 20 cases completed
[00:29.14] MainProcess/INFO: experiments finished


,input_flow,expand_capacity,no_build_travel_time,build_travel_time,time_savings,value_of_time_savings,net_benefits,cost_of_capacity_expansion,present_cost_expansion
experiment,,,,,,,,,
1,114,17.805981,75.200641,67.892100,7.308542,8.331738,-84.622494,92.954231,1780.598093
2,214,67.515416,248.754625,83.970577,164.784049,352.637864,0.180810,352.457054,6751.541583
3,166,10.068842,128.339982,106.560461,21.779522,36.154006,-16.409315,52.563321,1006.884225
4,199,83.864133,201.141528,72.349994,128.791534,256.295153,-181.508642,437.803796,8386.413311
5,236,74.413206,339.183997,90.169844,249.014154,587.673403,199.207168,388.466235,7441.320595


In [16]:
mm = m.create_metamodel_from_design('lhs', suppress_converge_warnings=True)
mm

<emat.PythonCoreModel "EMATMeta", metamodel_id=1 with 11 constants, 1 uncertainties, 1 levers, 7 measures>

In [17]:
mm_cv = mm.function.cross_val_scores()

In [18]:
mm_cv

no_build_travel_time          0.999959
build_travel_time             0.997855
time_savings                  0.694462
value_of_time_savings         0.779450
net_benefits                  0.949293
cost_of_capacity_expansion    0.472594
present_cost_expansion        0.472594
dtype: float64

In [19]:
lhs_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 1 to 20
Data columns (total 9 columns):
input_flow                    20 non-null int64
expand_capacity               20 non-null float64
no_build_travel_time          20 non-null float64
build_travel_time             20 non-null float64
time_savings                  20 non-null float64
value_of_time_savings         20 non-null float64
net_benefits                  20 non-null float64
cost_of_capacity_expansion    20 non-null float64
present_cost_expansion        20 non-null float64
dtypes: float64(8), int64(1)
memory usage: 1.6 KB


In [50]:
db.read_experiment_all(
    None,
    ['lhs', 'la_0', 'la_1', 'la_2'],
)

# db.read_design_names(None)

InterfaceError: Error binding parameter 1 - probably unsupported type.

In [37]:
maxval = numpy.fabs(lhs_results.net_benefits).max()

fig, (ax0, ax1) = plt.subplots(1,2, figsize=(12,5))

lhs_results.plot.scatter(
    ax=ax0,
    x='input_flow', 
    y='expand_capacity', 
    c='net_benefits',
    cmap='PiYG',
    vmax=maxval,
    vmin=-maxval,
) 

lhs_results.plot.scatter(
    ax=ax1,
    x='input_flow', 
    y='expand_capacity', 
    c='value_of_time_savings',
    cmap='viridis',
) 

Show(fig)

<xmle.Elem 'div' with 1 children>

In [21]:
Show(
)

<xmle.Elem 'div' with 1 children>

In [32]:
# px.scatter(
#     lhs_results,
#     x='input_flow', 
#     y='expand_capacity', 
#     color='value_of_time_savings',
#     #color_continuous_scale=px.colors.sequential.Viridis,
#     template='plotly_white+presentation',
# )

In [21]:
raise RuntimeError()

RuntimeError: 

### Loeppky Augmentation

In [38]:
n_augmentations = 3
aug_batch_size = 5
aug_design = {}
aug_results = {}
aug_mm = {-1: mm}
aug_cv = {-1: mm_cv}
aug_names = ['lhs',]

In [39]:
for aug in range(n_augmentations):
    
    design_pool = design_experiments(
        s, 
        n_samples=5000, 
        sampler='ulhs', 
        random_seed=42+aug,
    )
    
    aug_name = f'la_{aug}'
    aug_design[aug] = aug_mm[aug-1].function.pick_new_experiments(
            design_pool,
            batch_size=aug_batch_size,
            output_focus=dict(1-aug_cv[aug-1]),
            scope=s,
            db=db,
            design_name=aug_name,
    )
    aug_results[aug] = m.run_experiments(design_name=aug_name)
    aug_names.append(aug_name)
    aug_mm[aug] = m.create_metamodel_from_designs(aug_names, suppress_converge_warnings=True)
    aug_cv[aug] = aug_mm[aug].function.cross_val_scores()
    print(f"-- Cross Validation Loeppky {aug} --\n", aug_cv[aug])
        
#     j = aug_mm[aug].run_experiments(grid_df[['x1','x2']], db=False)
#     j_stderr = aug_mm[aug].function.compute_std(grid_df[['x1','x2']])
    
#     for cc in [1,2,3,4]:
#         fig, axs = plt.subplots(1,4, figsize=(13,3))
#         axs[0].scatter(grid_df.x1, grid_df.x2, c=grid_df[f'c{cc}'])
#         axs[0].set_title(f'Core{cc}')
#         axs[1].scatter(j.x1, j.x2, c=j[f'c{cc}'])
#         axs[1].set_title(f'Loeppky MetaModel{cc}')
#         diffs = grid_df[f'c{cc}']-j[f'c{cc}']
#         maxdiff = numpy.fabs(diffs).max()
#         axs[2].scatter(j.x1, j.x2, c=diffs, cmap='PiYG', vmin=-maxdiff, vmax=maxdiff)
#         axs[2].set_title(f'Error in MetaModel{cc}')

#         axs[3].scatter(j.x1, j.x2, c=j_stderr[f'c{cc}'])
#         axs[3].set_title(f'Std Err of MetaModel{cc}')

#         plt.show()



[13:38.26] MainProcess/INFO: selecting 3106
[13:38.35] MainProcess/INFO: selecting 1877
[13:38.46] MainProcess/INFO: selecting 937
[13:38.54] MainProcess/INFO: selecting 4125
[13:38.63] MainProcess/INFO: selecting 368
[13:38.99] MainProcess/INFO: 0 exchanges completed.
[13:39.01] MainProcess/INFO: performing 5 scenarios/policies * 1 model(s) = 5 experiments
[13:39.01] MainProcess/INFO: performing experiments sequentially
[13:39.02] MainProcess/INFO: 1 cases completed
[13:39.02] MainProcess/INFO: 2 cases completed
[13:39.02] MainProcess/INFO: 3 cases completed
[13:39.03] MainProcess/INFO: 4 cases completed
[13:39.03] MainProcess/INFO: 5 cases completed
[13:39.03] MainProcess/INFO: experiments finished
[14:03.50] MainProcess/INFO: selecting 1985


-- Cross Validation Loeppky 0 --
 no_build_travel_time          1.000000
build_travel_time             0.991103
time_savings                  0.514406
value_of_time_savings         0.612913
net_benefits                 -0.073555
cost_of_capacity_expansion    0.866231
present_cost_expansion        0.866231
dtype: float64


[14:03.63] MainProcess/INFO: selecting 4696
[14:03.72] MainProcess/INFO: selecting 2782
[14:03.81] MainProcess/INFO: selecting 1987
[14:03.92] MainProcess/INFO: selecting 4728
[14:04.34] MainProcess/INFO: 0 exchanges completed.
[14:04.36] MainProcess/INFO: performing 5 scenarios/policies * 1 model(s) = 5 experiments
[14:04.37] MainProcess/INFO: performing experiments sequentially
[14:04.37] MainProcess/INFO: 1 cases completed
[14:04.37] MainProcess/INFO: 2 cases completed
[14:04.38] MainProcess/INFO: 3 cases completed
[14:04.38] MainProcess/INFO: 4 cases completed
[14:04.38] MainProcess/INFO: 5 cases completed
[14:04.38] MainProcess/INFO: experiments finished
[14:28.38] MainProcess/INFO: selecting 148


-- Cross Validation Loeppky 1 --
 no_build_travel_time          0.999999
build_travel_time             0.997368
time_savings                  0.939806
value_of_time_savings         0.834140
net_benefits                  0.818605
cost_of_capacity_expansion    0.933649
present_cost_expansion        0.933649
dtype: float64


[14:28.48] MainProcess/INFO: selecting 2422
[14:28.62] MainProcess/INFO: selecting 3981
[14:28.72] MainProcess/INFO: selecting 2966
[14:28.80] MainProcess/INFO: selecting 3026
[14:29.18] MainProcess/INFO: 0 exchanges completed.
[14:29.19] MainProcess/INFO: performing 5 scenarios/policies * 1 model(s) = 5 experiments
[14:29.21] MainProcess/INFO: performing experiments sequentially
[14:29.21] MainProcess/INFO: 1 cases completed
[14:29.21] MainProcess/INFO: 2 cases completed
[14:29.21] MainProcess/INFO: 3 cases completed
[14:29.21] MainProcess/INFO: 4 cases completed
[14:29.22] MainProcess/INFO: 5 cases completed
[14:29.22] MainProcess/INFO: experiments finished


-- Cross Validation Loeppky 2 --
 no_build_travel_time          1.000000
build_travel_time             0.998588
time_savings                  0.955407
value_of_time_savings         0.971088
net_benefits                  0.928976
cost_of_capacity_expansion    0.930154
present_cost_expansion        0.749823
dtype: float64


In [ ]:
df = emat_db.read_experiment_all(scope_name=emat_db.read_scope_names()[0], design=None)

In [ ]:
(df.expand_capacity * df.unit_cost_expansion)

In [ ]:
df_p = emat_db.read_experiment_parameters(scope_name=emat_db.read_scope_names()[0], design=None)

In [ ]:
pandas.concat([
    df.present_cost_expansion.rename("CoreModel"),
    mm.function.predict(df_p, trend_only=True)['present_cost_expansion'].rename('Trend')
], axis=1)

In [ ]:
mm.function.predict(df_p, trend_only=True)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:

fig, ax = plt.subplots()
ax.scatter(
    df.unit_cost_expansion, 
    df.expand_capacity, 
    c=df.present_cost_expansion,
)
plt.show()
plt.close(fig)


In [ ]:
df.info()

In [ ]:
aug_mm[0].function.cross_val_scores()

In [ ]:
aug_mm[2].function.input_sample

In [ ]:
# aug_cv[aug] = aug_mm[aug].function.cross_val_scores()

